<a href="https://colab.research.google.com/github/atharv-d21/titanic-mL-model/blob/main/titanic_survival_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports for PyTorch
import torch
from torch import nn
torch.__version__

'2.6.0+cu124'

In [2]:
# import for other tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt